# Lokalizacja punktu w przestrzeni planarnej za pomocą metody trapezowej

### Konfiguracja narzędzia do wizualizacji algorytmu

In [850]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

FIG_EPS = 0.5

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax
        
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)
        
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))
         
    def add_line(self, event):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []
    
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < FIG_EPS:
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:    
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)
        
    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()


### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [851]:
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points = self.points + points

class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)
    
class Plot:
    def __init__(self, scenes = [Scene()], json = None):
        if json is None:
            self.scenes = scenes
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
        
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])    
     
    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None
  
    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None
        
    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None
    
    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None
    
    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()
        

# Rozwiązanie

## Potrzebne klasy i struktury danych

In [852]:
%matplotlib notebook

def count_ab(segment):
    if segment.end.x - segment.start.x == 0:
        return 0, 0
    return ((segment.end.y - segment.start.y) / (segment.end.x - segment.start.x),
            segment.start.y - segment.start.x * ((segment.end.y - segment.start.y) / (segment.end.x - segment.start.x)))


def count_y(segment, x):
    a, b = count_ab(segment)
    return a * x + b

class Point:
    def __init__(self, point):
        self.x = point[0]
        self.y = point[1]
    
    def draw(self):
        return [self.x, self.y]
    
    def __str__(self):
        return f'({self.x},{self.y})'

class Segment:
    def __init__(self, segment):
        self.segment = segment
        if segment[0][0] < segment[1][0]:
            self.start = Point(segment[0])
            self.end = Point(segment[1])
        else:
            self.start = Point(segment[1])
            self.end = Point(segment[0])
    
    def draw(self):
        return [self.start.draw(), self.end.draw()]
    
    def __str__(self):
        return f'[{str(self.start)}, {str(self.end)}]'
    
    
class Trapezoid:
    def __init__(self, leftp, rightp, top, bottom, left_top = None, left_bottom = None, right_bottom = None, right_top = None):
        self.leftp = leftp
        self.rightp = rightp
        self.top = top
        self.bottom = bottom
        self.left_top = left_top
        self.left_bottom = left_bottom
        self.right_bottom = right_bottom
        self.right_top = right_top
        self.node = None
        
    def draw(self):
        res = []
        res.append((self.leftp.x, count_y(self.bottom, self.leftp.x)))
        res.append((self.rightp.x, count_y(self.bottom, self.rightp.x)))
        res.append((self.rightp.x, count_y(self.top, self.rightp.x)))
        res.append((self.leftp.x, count_y(self.top, self.leftp.x)))
        res.append(res[0])
        return [[[res[i][0], res[i][1]], [res[i + 1][0], res[i + 1][1]]] for i in range(len(res)-1)]

WALL = 0 # trapezoid
X_NODE = 1 # Point 
Y_NODE = 2 # Segment

def det(a,b,c):
    return a.x*b.y + a.y*c.x + b.x*c.y - b.y*c.x - a.y*b.x - a.x*c.y

def is_above_segment(point, segment, eps=1e-6):
    orient = det(segment.start, segment.end, point)
    return orient > eps


class Node:
    def __init__(self, type, value, left = None, right = None):
        if type == WALL:
            value.node = self
        self.type = type
        self.value = value
        self.left = left
        self.right = right

    
class Graph:
    def __init__(self, starting_node = None):
        self.starting_node = starting_node
    
    def find(self, point_to_find, scenes = [], segments = []):
        if scenes:
            return self._find(point_to_find, self.starting_node, scenes, segments)
        return self._find(point_to_find, self.starting_node)
    
    def _find(self, point_to_find, current_node, scenes = [], segments = []):
        if current_node is None: return -1
        if current_node.type == WALL:
            if scenes:
                scenes.append(
                    Scene(
                        [PointsCollection([point_to_find.draw()], color="red")],
                        [LinesCollection(draw_collection(segments)),
                         LinesCollection(current_node.value.draw(), color="orange")]
                    )
                )
            return current_node.value
        elif current_node.type == X_NODE:
            if scenes:
                scenes.append(
                    Scene(
                        [PointsCollection([current_node.value.draw()], color="orange"),
                         PointsCollection([point_to_find.draw()], color="red")],
                        [LinesCollection(draw_collection(segments))]
                    )
                )
            if point_to_find.x < current_node.value.x:
                return self._find(point_to_find, current_node.left, scenes, segments)
            else:
                return self._find(point_to_find, current_node.right, scenes, segments)
        elif current_node.type == Y_NODE:
            if scenes:
                scenes.append(
                    Scene(
                        [PointsCollection([point_to_find.draw()], color="red")],
                        [LinesCollection(draw_collection(segments)),
                         LinesCollection([current_node.value.draw()], color="orange")]
                    )
                )
            if is_above_segment(point_to_find, current_node.value):
                return self._find(point_to_find, current_node.left, scenes, segments)
            else:
                return self._find(point_to_find, current_node.right, scenes, segments)
    
    def replace_node(self, old_node, new_node):
        old_node.type = new_node.type
        old_node.value = new_node.value
        old_node.right = new_node.right
        old_node.left = new_node.left

    def get_trapezoids(self, node, trapezoid_list):
        if node.type == WALL:
            trapezoid_list.extend(node.value.draw())
            return
        self.get_trapezoids(node.left, trapezoid_list)
        self.get_trapezoids(node.right, trapezoid_list)
        

### Funkcje pomocniczne

In [853]:
%matplotlib notebook

def prepare_data(segments):
    new_segments = []
    MAX_X = MAX_Y = 0
    MIN_X = MIN_Y = 20
    for segment in segments:
        new_segment = Segment(segment)
        new_segments.append(new_segment)
        for point in [new_segment.start, new_segment.end]:
            MIN_Y = min(MIN_Y, point.y)
            MIN_X = min(MIN_X, point.x)
            MAX_Y = max(MAX_Y, point.y)
            MAX_X = max(MAX_X, point.x)

    return new_segments, MAX_Y+1, MIN_Y-1, MAX_X+1, MIN_X-1
        

def draw_collection(elements):
    return [ element.draw() for element in elements]

# Narysuj odcinki lub wielokąty tworzące mapę planarną

In [854]:
plot_to_draw = Plot([Scene([PointsCollection([[0,0], [20, 20]])],[])])
plot_to_draw.draw()

<IPython.core.display.Javascript object>

### Zapisywanie to do formatu json

In [855]:
%matplotlib notebook

figure_from_previous_plot = plot_to_draw.get_added_figure()
segments_from_previous_plot = plot_to_draw.get_added_lines()
scenes = [Scene([], figure_from_previous_plot),
          Scene([], segments_from_previous_plot)]
with open('plot.json', 'w') as file:
    file.write(Plot(scenes).toJson())

### Odczytanie danych wejściowych z pliku json

In [856]:
%matplotlib notebook

with open('plot.json', 'r') as file:
    json = file.read()
helper_plot = Plot(json=json)
elements = helper_plot.scenes[0].lines and helper_plot.scenes[0].lines[0].lines
lines = helper_plot.scenes[1].lines and helper_plot.scenes[1].lines[0].lines
segments = elements + lines

### Przygotowanie danych

In [857]:
%matplotlib notebook

new_segments, MAX_Y, MIN_Y, MAX_X, MIN_X = prepare_data(segments)

scenes = [Scene(
    [],
    [LinesCollection(draw_collection(new_segments)),
     LinesCollection(Trapezoid(
        Point((MIN_X, MIN_Y)),
        Point((MAX_X, MAX_Y)),
        Segment([(MIN_X, MAX_Y), (MAX_X, MAX_Y)]),
        Segment([(MIN_X, MIN_Y), (MAX_X, MIN_Y)])
        ).draw(), color="green")])]
plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

In [858]:
import copy
  
def update_graph_first_segment(trapezoid, segment, D):
    p = segment.start
    q = segment.end
    old_node = trapezoid.node
    p_node = Node(X_NODE, p)
    q_node = Node(X_NODE, q)
    s_node = Node(Y_NODE, segment)
    trapezoid_left = Node(WALL, trapezoid)
    trapezoid_top = Node(WALL, trapezoid.right_top)
    trapezoid_bottom = Node(WALL, trapezoid.right_bottom)
    trapezoid_right = Node(WALL, trapezoid.right_bottom.right_top)
    p_node.left = trapezoid_left
    p_node.right = q_node
    q_node.left = s_node
    q_node.right = trapezoid_right
    s_node.left = trapezoid_top
    s_node.right = trapezoid_bottom
    D.replace_node(old_node, p_node)


def update_graph_first_element(trapezoid, segment, D):
    p = segment.start
    old_node = trapezoid.node
    p_node = Node(X_NODE, p)
    s_node = Node(Y_NODE, segment)
    trapezoid_left = Node(WALL, trapezoid)
    trapezoid_top = Node(WALL, trapezoid.right_top)
    trapezoid_bottom = Node(WALL, trapezoid.right_bottom)
    p_node.left = trapezoid_left
    p_node.right = s_node
    s_node.left = trapezoid_top
    s_node.right = trapezoid_bottom
    D.replace_node(old_node, p_node)
    
    
def update_graph_last_element(trapezoid, segment, D):
    q = segment.end
    old_node = trapezoid.node
    q_node = Node(X_NODE, q)
    s_node = Node(Y_NODE, segment)
    trapezoid_right = Node(WALL, trapezoid)
    trapezoid_top = Node(WALL, trapezoid.left_top)
    trapezoid_bottom = Node(WALL, trapezoid.left_bottom)
    q_node.right = trapezoid_right
    q_node.left = s_node
    s_node.left = trapezoid_top
    s_node.right = trapezoid_bottom
    D.replace_node(old_node, q_node)


def update_graph_middle_element(top_trapezoid, bot_trapezoid, old_trapezoid, segment, D):
    old_node = old_trapezoid.node
    s_node = Node(Y_NODE, segment)
    trapezoid_top = Node(WALL, top_trapezoid)
    trapezoid_bottom = Node(WALL, bot_trapezoid)
    s_node.left = trapezoid_top
    s_node.right = trapezoid_bottom
    D.replace_node(old_node, s_node)
###################################################################################  

def update_res(res, list_of_trapezoids):
    for trapezoid in list_of_trapezoids:
        res.extend(trapezoid.draw())
        
###################################################################################

# Updating trapezoid map
def first_segment_insertion(trapezoid, segment, res):
    end = trapezoid.rightp
    trapezoid.rightp = segment.start
    new_trapezoid1 = Trapezoid(segment.start, segment.end, trapezoid.top, segment, trapezoid, trapezoid)
    new_trapezoid2 = Trapezoid(segment.start, segment.end, segment, trapezoid.bottom, trapezoid, trapezoid)
    new_trapezoid3 = Trapezoid(segment.end, end, trapezoid.top, trapezoid.bottom, new_trapezoid1, new_trapezoid2, trapezoid.right_top, trapezoid.right_bottom)
    new_trapezoid1.right_top = new_trapezoid1.right_bottom = new_trapezoid3
    new_trapezoid2.right_top = new_trapezoid2.right_bottom = new_trapezoid3
    trapezoid.right_top = new_trapezoid1
    trapezoid.right_bottom = new_trapezoid2
    update_res(res, [trapezoid, new_trapezoid1, new_trapezoid2, new_trapezoid3])


def update_first_element(trapezoid, segment, res):
    end = trapezoid.rightp
    trapezoid.rightp = segment.start
    if is_above_segment(end, segment):
        new_trapezoid1 = Trapezoid(segment.start, end, trapezoid.top, segment, trapezoid, trapezoid, trapezoid.right_bottom, trapezoid.right_top)
        new_trapezoid2 = Trapezoid(segment.start, None, segment, trapezoid.bottom, trapezoid, trapezoid, trapezoid.right_bottom, trapezoid.right_bottom)
        update_res(res, [trapezoid, new_trapezoid1])
    else:
        new_trapezoid1 = Trapezoid(segment.start, None, trapezoid.top, segment, trapezoid, trapezoid, trapezoid.right_top, trapezoid.right_top)
        new_trapezoid2 = Trapezoid(segment.start, end, segment, trapezoid.bottom, trapezoid, trapezoid, trapezoid.right_bottom, trapezoid.right_top)
        update_res(res, [trapezoid, new_trapezoid2])
    trapezoid.right_top = new_trapezoid1
    trapezoid.right_bottom = new_trapezoid2
    return end


def update_last_element(prev_top, prev_bot, trapezoid, segment, res):
    start = trapezoid.leftp
    trapezoid.leftp = segment.end        
    if is_above_segment(start, segment):          
        new_trapezoid = Trapezoid(start, segment.end, trapezoid.top, segment, None, prev_top, trapezoid, trapezoid)
        if trapezoid.left_top == trapezoid.left_bottom: # has only one left sibling
            new_trapezoid.left_top = prev_top
        else:
            new_trapezoid.left_top = trapezoid.left_top
            trapezoid.left_top.right_bottom = new_trapezoid
        prev_top.right_top = prev_top.right_bottom = new_trapezoid
        trapezoid.left_top = new_trapezoid
        prev_bot.rightp = segment.end
        trapezoid.left_bottom = prev_bot
        update_res(res, [trapezoid, new_trapezoid, prev_bot])
    else:
        new_trapezoid = Trapezoid(start, segment.end, segment, trapezoid.bottom, prev_bot, None, trapezoid, trapezoid)
        if trapezoid.left_top == trapezoid.left_bottom: # has only one left sibling
            new_trapezoid.left_bottom = prev_bot
        else:
            new_trapezoid.left_bottom = trapezoid.left_bottom
            trapezoid.left_bottom.right_top = new_trapezoid
        prev_bot.right_top = prev_bot.right_bottom = new_trapezoid
        trapezoid.left_bottom = new_trapezoid
        prev_top.rightp = segment.end
        trapezoid.left_top = prev_top
        update_res(res, [trapezoid, new_trapezoid, prev_top])

    return start


def update_middle_element(prev_top, prev_bot, trapezoid_to_update, segment, res, D):
    if trapezoid_to_update.leftp.x < segment.end.x < trapezoid_to_update.rightp.x:
        update_last_element(prev_top, prev_bot, trapezoid_to_update, segment, res)
        update_graph_last_element(trapezoid_to_update, segment, D)
        return
    start = trapezoid_to_update.leftp
    end = trapezoid_to_update.rightp
    if is_above_segment(start, segment):
        if is_above_segment(end, segment):
            #oba na górze
            prev_bot.right_top = prev_bot.right_bottom = trapezoid_to_update.right_bottom
            trapezoid_to_update.bottom = segment
            update_res(res, [trapezoid_to_update])
            update_graph_middle_element(trapezoid_to_update, prev_bot, trapezoid_to_update, segment, D)
            update_middle_element(trapezoid_to_update, prev_bot, prev_bot.right_top, segment, res, D)
        else:
            #start na górze, koniec na dole
            prev_bot.rightp = end
            prev_bot.right_top = trapezoid_to_update.right_top
            prev_bot.right_bottom = trapezoid_to_update.right_bottom
            trapezoid_to_update.rightp = None
            trapezoid_to_update.bottom = segment
            trapezoid_to_update.right_bottom = trapezoid_to_update.right_top
            update_res(res, [prev_bot])
            update_graph_middle_element(trapezoid_to_update, prev_bot, trapezoid_to_update, segment, D)
            update_middle_element(trapezoid_to_update, prev_bot, trapezoid_to_update.right_bottom, segment, res, D)
    else:
        if is_above_segment(end, segment):
            # start na dole, koniec na górze
            prev_top.rightp = end
            prev_top.right_top = trapezoid_to_update.right_top
            prev_top.right_bottom = trapezoid_to_update.right_bottom
            trapezoid_to_update.rightp = None
            trapezoid_to_update.top = segment
            trapezoid_to_update.right_top = trapezoid_to_update.right_bottom
            update_res(res, [prev_top])
            update_graph_middle_element(prev_top, trapezoid_to_update, trapezoid_to_update, segment, D)
            update_middle_element(prev_top, trapezoid_to_update, trapezoid_to_update.right_top, segment, res, D)            
        else:
            #oba na dole
            prev_top.right_top = prev_top.right_bottom = trapezoid_to_update.right_top
            trapezoid_to_update.top = segment
            update_res(res, [trapezoid_to_update])
            update_graph_middle_element(prev_top, trapezoid_to_update, trapezoid_to_update, segment, D)
            update_middle_element(prev_top, trapezoid_to_update, prev_top.right_bottom, segment, res, D) 
    

def new_segment_in_map(trapezoid, segment, res, D):
    if trapezoid.leftp.x < segment.start.x and trapezoid.rightp.x > segment.end.x:
        first_segment_insertion(trapezoid, segment, res)
        update_graph_first_segment(trapezoid, segment, D)
        return
    update_first_element(trapezoid, segment, res)
    update_graph_first_element(trapezoid, segment, D)
    update_middle_element(trapezoid.right_top, trapezoid.right_bottom, trapezoid.right_top.right_bottom, segment, res, D)

# Główny algorytm

In [859]:
import random

def create_trapezoid_map_and_search_graph(segments, MAX_Y, MIN_Y, MAX_X, MIN_X):
    random.shuffle(segments)
    rectangle = Trapezoid(
        Point((MIN_X, MIN_Y)),
        Point((MAX_X, MAX_Y)),
        Segment([(MIN_X, MAX_Y), (MAX_X, MAX_Y)]),
        Segment([(MIN_X, MIN_Y), (MAX_X, MIN_Y)])
        )
    scenes = [Scene([], [LinesCollection(rectangle.draw())])]
    starting_node = Node(WALL, rectangle)
    D = Graph(starting_node)
    added_segments_map = []
    for segment in segments:
        try:
            graphical_response = []
            found_trapezoid = D.find(segment.start)
            new_segment_in_map(found_trapezoid, segment, graphical_response, D)
        except:
            added_segments_map.append(
                segment.draw()
            )
            scenes.append(
                Scene(
                    [],
                    [LinesCollection(copy.deepcopy(graphical_response), color="green"),
                     LinesCollection(copy.deepcopy(added_segments_map)),
                    ]
                )
            )
            return D, scenes
        added_segments_map.append(
            segment.draw()
        )
        scenes.append(
            Scene(
                [],
                [LinesCollection(copy.deepcopy(graphical_response), color="green"),
                 LinesCollection(copy.deepcopy(added_segments_map))
                 ]
            )
        )
    return D, scenes

## Prezentacja działania

In [860]:
%matplotlib notebook

D, scenes = create_trapezoid_map_and_search_graph(new_segments, MAX_Y, MIN_Y, MAX_X, MIN_X)
plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

## Prezentacja wyszukiwania w stworzonym grafie wyszukiwań
### Zaznacz punkt do znalezienia

In [861]:
plot_to_draw = Plot([Scene([],[LinesCollection(draw_collection(new_segments))])])
plot_to_draw.draw()

<IPython.core.display.Javascript object>

# Wizualizacja znajdywania w grafie

In [862]:
%matplotlib notebook
points = plot_to_draw.get_added_points()[0]
point_to_find = Point(points.points[0])
scenes = [Scene([PointsCollection([point_to_find.draw()], color="red")],
                [LinesCollection(draw_collection(new_segments))]
                )]
found_trapezoid = D.find(point_to_find, scenes, new_segments)
plot = Plot(scenes)
plot.draw()

IndexError: list index out of range